# Posture detection using tensorflow判斷人體姿態<br>**Bonus**: Step 7

##### We have successfully implemented the Squat Counting function in Step 6 just now. <br>It looks good, but are there still some defects? <br>剛剛我們在Step6中，成功運行到Squat Counting的功能。<br>這樣看起來很不錯，但是是否仍然有一些缺陷？<br><br>Let's evaluate our Step 6 code again in terms of speed:<br>讓我們在速度上再次評估我們Step6的代碼: 

In [1]:
from step6 import *
from step6 import _keypoints_and_edges_for_display

====== Importing from step 3 ... ======
Libraries imported.
Model loaded.
Image loaded.

The following functions have been imported:
   movenet(input_image)
   _keypoints_and_edges_for_display(keypoints_with_scores, height, width, keypoint_threshold=0.11)
   draw_prediction_on_image(image, keypoints_with_scores, crop_region=None, close_figure=False, output_image_height=None)
The following global variables have been assigned:
   module, model_name, input_size, KEYPOINT_DICT, KEYPOINT_EDGE_INDS_TO_COLOR

====== Importing from step 4 ... ======

The following functions have been imported:
   load_image_to_tf(image)
   tf_image_to_model(image, width, height)
   get_keypoints_with_scores_from_image_with_movenet(image, width, height)
   show_image(image)
    
====== Importing from step 5 ... ======

The following functions have been imported:
   get_keypoints_and_edges_from_image_with_movenet(image, width, height)
   draw_output_from_image_with_movenet(image)
   view( button, source=0, rotat

In [4]:
import time
time_elapsed = []

def frame_process(frame):
    global repetition, squat, squatting
    # calculate and display angle    
    
    ''' mark the start time of MoveNet Conversion'''
    start_time = time.time()
    
    keypoints_with_scores = get_keypoints_with_scores_from_image_with_movenet(frame,input_size,input_size)
    
    ''' mark the end time of MoveNet Conversion'''
    ''' endtime - start_time is the time elapsed for MoveNet to process a frame'''
    time_elapsed.append( time.time() - start_time )
    
    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)
    # check if confidence is qualified, if yes, proceed to judgement
    if hip_angle is not None and knee_angle is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle<125 and knee_angle<105:
            if squat == 0:
                squatting += 1
            else:
                squatting = CONSECUTIVE_TH
        elif hip_angle>145 and knee_angle>125:
            if squat == 1:
                squatting -= 1
            else:
                squatting = 0
        if squatting == 0 and squat == 1:
            repetition += 1
            squat = 0
        elif squatting == CONSECUTIVE_TH and squat == 0:
            squat = 1
        text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    else:
        text = f"Reps: {repetition}  Knee: ?  Hip: ?"
    if squat:
        text_color = (0,255,0)
    else:
        text_color = (128,192,64)
    putText(frame, text, text_color)
    return frame

if __name__ == "__main__":
    display(stopButton)
    thread = threading.Thread(target=view, args=(stopButton, "Produce_2.mp4", 0, frame_process))
    thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

In [5]:
average_time = sum(time_elapsed) / len(time_elapsed)

print(f"Average running time of Movenet is: { average_time }")
print(f"Average frame rate of Movenet is: { 1 / average_time }")

Average running time of Movenet is: 0.12882413014326946
Average frame rate of Movenet is: 7.762520879340445


##### <br><hr> After completing the test, please restart the IPython Kernel and do not run any of the cells above again.<br>完成測試後，請重啓IPython Kernel，並**不要再運行**上面的任何單元格
<br>![img7-1.png](./resource/img7-1.png)
<br><hr>
Although our Step 6 code can achieve the basic function of squat counting, the slow inference of Movenet causes our counter to run less smoothly and responsively. <br>
<br>
Movenet is a complex deep learning model that requires a large amount of computational resources. The CPU and memory of Raspberry Pi 4 are limited, and cannot provide enough resources to run Movenet in real-time.
可以看出，雖然我們Step6的代碼能夠實現深蹲計數的基本功能，但由於Movenet的推理太過緩慢，使得我們的計數器運行不夠流暢和靈敏。<br>
Movenet是一種複雜的深度學習模型，需要大量的計算資源，而Raspberry Pi 4的處理器和內存容量有限，無法提供足夠的計算資源來實時運行Movenet。<br>
<br>
The Coral accelerator offers a solution for this problem. It's designed to speed up deep learning inference and can be used with Raspberry Pi 4. It boosts the model inference speed, enabling smoother and real-time operation of Movenet on Raspberry Pi 4.<br>
而針對這個問題，Coral accelerator可以提供一個解決方案。Coral accelerator是一種專門設計用於加速深度學習推理的硬體加速器，可以與Raspberry Pi 4一起使用。Coral accelerator可以大大提高模型推理速度，從而使得在Raspberry Pi 4上運行Movenet變得更加流暢和實時。<br>

##### <hr>First, we import the necessary libraries and redefine some of the functions used before.<br>首先，我們導入所需要的庫，並重新定義一些之前的函數

In [1]:
from IPython.display import display, Image
import IPython
import ipywidgets as widgets
import threading
import cv2
import numpy as np

def putText(frame, text, color = (0, 255, 0)):
    # Define the text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_position = (50, 50)
    text_scale = 0.65
    text_color = color
    text_thickness = 2

    # Add text annotation on the frame
    cv2.putText(frame, text, text_position, font, text_scale, text_color, text_thickness)

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

def calculate_hip_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 
    
    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_shoulder"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_shoulder"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None



def calculate_knee_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 

    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_ankle"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_ankle"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None


stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Display function
# ================
def view( button, source=0, rotate=0, process=(lambda x:x) ):
    """
    Args:
    button: An IPywidget button to control the function. The display stops if button.value == True.
    Source: An optional integer or filename to specify the source of the video stream.
    Rotate: optional integer. Set to 0 by default (not rotated). Set to 1 will rotate by 90 deg.
    process: optional function that processes each frame through process(frame) before displaying.
        Set to identity function by default.
    """
    display_handle=display("Please wait...", display_id=True)
    
    cap = cv2.VideoCapture(source)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
    
    while True:
        ret, frame = cap.read()
        if button.value==True or not ret:
            cap.release()
            display_handle.update(None)
            button.value = False
            break
        # frame = cv2.flip(frame, 1) # if your camera reverses your image

        # Rotate the frame if rotate==1
        if rotate:
            frame = cv2.transpose(frame)
            frame = cv2.flip(frame, 0)
        
        frame = process(frame)
        
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPython.display.Image(data=frame.tobytes()))
        

##### Next, let's try using Python to drive the Coral Accelerator to perform image inference.<br>接下來，讓我們試着用python驅動Coral Accelerator去執行圖片推理<br>
After importing the relevant libraries, we define the `get_result_from_image_with_model`. This function takes an image as input and applies a pre-trained pose estimation model to extract the positions of human keypoints in the image.<br>
導入相關庫之後，我們定義了`get_result_from_image_with_model`函數。這個函數需要輸入一張圖像並應用預先訓練的姿勢判斷模型，以提取圖像中人體關鍵點的位置。<br>
With this function, we can use the Coral Accelerator to quickly obtain the `keypoints_with_scores` result by providing an image and an interpreter. Moreover, the code is much shorter than before!<br>
這樣一來，我們就可以利用這個函數，給定一張圖像和解釋器，就能夠利用Coral Accelerator快速得出keypoints_with_scores的結果了！而且比之前我們的代碼簡短了許多！

In [3]:
import PIL
from PIL import Image
from PIL import ImageDraw
from pycoral.adapters import common
from pycoral.utils.edgetpu import make_interpreter

interpreter = make_interpreter("movenet_single_pose_lightning_ptq_edgetpu.tflite")
interpreter.allocate_tensors()

_NUM_KEYPOINTS = 17

def get_result_from_image_with_model(input, interpreter, output=None):
    '''
    This function takes an input image and applies a pre-trained pose estimation model 
    to extract the position of human body keypoints in the image. 
    The TensorFlow Lite interpreter object is used to run inference on the input image
    using the pre-trained model.
    If the optional output argument is provided, the function also saves the input image
    with the detected keypoints drawn on it to the specified file path.

    Args:
    input: An input image file path or a numpy array representing an image.
    interpreter: A TensorFlow Lite interpreter object.
    output (optional): A file path to save the output image with keypoints drawn on it.

    Return:
    Array of keypoints. Each keypoint is represented as a triplet of [y, x, score].
    '''
    # load the input image using PIL
    if isinstance(input, str):
        img = PIL.Image.open(input)
    elif isinstance(input, np.ndarray):
        img = PIL.Image.fromarray(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))

    # resize the image to model required size
    resized_img = img.resize(common.input_size(interpreter), PIL.Image.Resampling.LANCZOS)

    # load the resized image to interpreter
    common.set_input(interpreter, resized_img)

    # conduct the inference
    interpreter.invoke()

    # reshape and assign the inference result to variable `pose`
    keypoints_with_scores = common.output_tensor(interpreter, 0).copy().reshape(_NUM_KEYPOINTS, 3)

    # draw the keypoints and save the image (if specified `output`)
    if output:
        draw = ImageDraw.Draw(img)
        width, height = img.size
        for i in range(0, _NUM_KEYPOINTS):
            draw.ellipse(
            xy=[
                keypoints_with_scores[i][1] * width - 2, keypoints_with_scores[i][0] * height - 2,
                keypoints_with_scores[i][1] * width + 2, keypoints_with_scores[i][0] * height + 2
            ],
            fill=(255, 0, 0))
        img.save(output)
        
    return keypoints_with_scores

if __name__ == "__main__":
    keypoints_with_scores = get_result_from_image_with_model("input_image.jpeg", interpreter)
    print(type(keypoints_with_scores), keypoints_with_scores.shape, keypoints_with_scores, sep="\n\n")

<class 'numpy.ndarray'>

(17, 3)

[[0.32774833 0.58175325 0.36462   ]
 [0.3113609  0.589947   0.70056206]
 [0.3113609  0.5694627  0.5694627 ]
 [0.31955463 0.5735596  0.29907033]
 [0.3113609  0.50391304 0.49981618]
 [0.42197597 0.5571721  0.49981618]
 [0.40558854 0.43016967 0.49981618]
 [0.5121068  0.70875573 0.29907033]
 [0.5080099  0.53668785 0.8029834 ]
 [0.42197597 0.6841746  0.5694627 ]
 [0.4096854  0.65549666 0.19664899]
 [0.6022375  0.38100743 0.70056206]
 [0.6022375  0.28268293 0.70056206]
 [0.6473029  0.58175325 0.43016967]
 [0.6841746  0.41378227 0.43016967]
 [0.83166134 0.5162036  0.24581124]
 [0.88901734 0.33184516 0.75382113]]


##### Try it out: fill in the blank below with a Python statement that runs the hardware accelerated model, and then run the code.<br>試一試：在下面的空缺處填上一個運行硬件加速模型的python語句，然後運行代碼。<br>
Can you run the Step 6 code and see if it works? Check the speed in the last cell.<br>
你能運行出Step6的效果嗎？在最後的單元格看看速度如何？

In [8]:
import time
time_elapsed = []


repetition = squat = squatting = 0
CONSECUTIVE_TH=3
def frame_process(frame):
    global repetition, squat, squatting
    
    # calculate and display angle
    start_time = time.time()
    #   ====== ⇩ ⇩ ⇩  write your answer here   ⇩ ⇩ ⇩ ======
    
    keypoints_with_scores = get_result_from_image_with_model(frame, interpreter)
    #  ====== ⇧ ⇧ ⇧  write your answer here  ⇧ ⇧ ⇧ ======
    time_elapsed.append( time.time() - start_time )
    
    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    # check if confidence is qualified, if yes, proceed to judgement
    if hip_angle is not None and knee_angle is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle<125 and knee_angle<105:
            if squat == 0:
                squatting += 1
            else:
                squatting = CONSECUTIVE_TH
        elif hip_angle>145 and knee_angle>125:
            if squat == 1:
                squatting -= 1
            else:
                squatting = 0
        if squatting == 0 and squat == 1:
            repetition += 1
            squat = 0
        elif squatting == CONSECUTIVE_TH and squat == 0:
            squat = 1
        text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    else:
        text = f"Reps: {repetition}  Knee: ?  Hip: ?"
    if squat:
        text_color = (0,255,0)
    else:
        text_color = (128,192,64)
        
    putText(frame, text, text_color)
    return frame


# Run
# ================
if __name__ == "__main__":
    display(stopButton)
    thread = threading.Thread(target=view, args=(stopButton, "Produce_2_direct.mp4", 0, frame_process))
    thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

In [ ]:
average_time = sum(time_elapsed) / len(time_elapsed)

print(f"Average running time of Movenet is: { average_time }")
print(f"Average frame rate of Movenet is: { 1 / average_time }")